In [1]:
import anndata
import numpy as np
import scvelo as scv
import scanpy as sc
import sys
import torch
import os.path
import deepvelo as dv
import pickle as pickle
import matplotlib.pyplot as plt
import pandas as pd
import unitvelo as utv
from os.path import exists
method = 'DeepVelo'

(Running UniTVelo 0.2.5)
2023-05-16 16:18:42


In [2]:
datasets = ['Pancreas_with_cc', 'HumanDevelopingBrain', 'DentateGyrus' , 'MouseBoneMarrow', 'MouseErythroid', 'HumanBoneMarrow']
data_dir = '/nfs/team283/aa16/data/fate_benchmarking/benchmarking_datasets/'
save_dir = '/nfs/team283/aa16/data/fate_benchmarking/benchmarking_results/'

In [ ]:
for dataset in datasets:
    print(dataset)
    adata = sc.read_h5ad(data_dir + dataset + '/' + dataset + '_anndata.h5ad')
    scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=3000)
    scv.pp.moments(adata, n_pcs=30, n_neighbors=30)
    trainer = dv.train(adata, dv.Constants.default_configs)
    scv.pp.neighbors(adata)
    scv.tl.velocity_graph(adata, vkey = 'velocity')
    scv.tl.velocity_embedding(adata, vkey = 'velocity')
    fix, ax = plt.subplots(1, 1, figsize = (8, 6))
    scv.pl.velocity_embedding_stream(adata, basis='umap', save = False, vkey='velocity',
                                     show = False, ax = ax)
    plt.savefig(save_dir + 'UMAPs/' + dataset + '_UMAP_DeepVelo.png')
    # Calculate performance metrics:
    file = open(data_dir + dataset + '/' + dataset + '_groundTruth.pickle' ,'rb')
    ground_truth = pickle.load(file)
    metrics = utv.evaluate(adata, ground_truth, 'clusters', 'velocity')
    if exists(save_dir + dataset + '_CBDC_scores.csv'):
        tab = pd.read_csv(save_dir + dataset + '_CBDC_scores.csv', index_col = 0)
    else:
        tab = pd.DataFrame(columns = list(metrics['Cross-Boundary Direction Correctness (A->B)'].keys()) + ['Mean'],
                 index = [method])
    cb_score = [np.mean(metrics['Cross-Boundary Direction Correctness (A->B)'][x])
                for x in metrics['Cross-Boundary Direction Correctness (A->B)'].keys()]
    tab.loc[method,:] = cb_score + [np.mean(cb_score)]
    tab.to_csv(save_dir + dataset + '_CBDC_scores.csv')
    metrics = utv.evaluate(adata, ground_truth, 'clusters', 'velocity')
    if exists(save_dir + dataset + '_ICC_scores.csv'):
        tab = pd.read_csv(save_dir + dataset + '_ICC_scores.csv', index_col = 0)
    else:
        tab = pd.DataFrame(columns = list(np.unique(np.concatenate(ground_truth))) + ['Mean'],
                 index = [method])
    icc_score = [np.mean(metrics['In-cluster Coherence'][x]) for x in np.unique(np.concatenate(ground_truth))]
    tab.loc[method,:] = icc_score + [np.mean(icc_score)]
    tab.to_csv(save_dir + dataset + '_ICC_scores.csv')
    fix, ax = plt.subplots(1, 1, figsize = (8, 6))
    scv.pl.velocity_embedding_stream(adata, basis='umap', save = False, vkey='velocity',
                                     show = False, ax = ax)
    plt.savefig(save_dir + 'UMAPs/' + dataset + '_UMAP_' + method + '.png')

Pancreas_with_cc
Filtered out 20801 genes that are detected 20 counts (shared).
Normalized count data: X, spliced, unspliced.
Extracted 3000 highly variable genes.
Logarithmized X.
computing neighbors
    finished (0:00:07) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:00) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)
Config Warning: Set to use GPU, but GPU version of DGL is not installed. Reset to use CPU instead.
building graph


INFO:train:Beginning training of DeepVelo_Base ...


velo data shape: torch.Size([3696, 3000])


/nfs/team283/aa16/software/miniconda3/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/nfs/team283/aa16/software/miniconda3/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):
INFO:trainer:    epoch          : 1
INFO:trainer:    time:          : 10.06196141242981
INFO:trainer:    loss           : 98173.234375
INFO:trainer:    mse            : 5.441349029541016
INFO:trainer:    epoch          : 2
INFO:trainer:    time:          : 10.080729246139526
INFO:trainer:    loss           : 15773.8271484375
INFO:trainer:    mse            : 2.0281200408935547
INFO:trainer:    epoch          : 3
INFO:trainer:    time:          : 10.069921970367432
INFO:trainer:    loss           : 9049.268554

INFO:trainer:    epoch          : 41
INFO:trainer:    time:          : 10.174213171005249
INFO:trainer:    loss           : 2181.993896484375
INFO:trainer:    mse            : 0.4727761447429657
INFO:trainer:    epoch          : 42
INFO:trainer:    time:          : 10.103241682052612
INFO:trainer:    loss           : 2171.04833984375
INFO:trainer:    mse            : 0.4580151438713074
INFO:trainer:    epoch          : 43
INFO:trainer:    time:          : 10.206052303314209
INFO:trainer:    loss           : 2164.229736328125
INFO:trainer:    mse            : 0.43244290351867676
INFO:trainer:    epoch          : 44
INFO:trainer:    time:          : 10.100700616836548
INFO:trainer:    loss           : 2159.19140625
INFO:trainer:    mse            : 0.4359542429447174
INFO:trainer:    epoch          : 45
INFO:trainer:    time:          : 10.132095575332642
INFO:trainer:    loss           : 2156.2880859375
INFO:trainer:    mse            : 0.4474633038043976
INFO:trainer:    epoch         

...

  0%|          | 0/2600 [00:00<?, ?cells/s]

    finished (0:00:05) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)
computing velocity embedding
    finished (0:00:00) --> added
    'velocity_umap', embedded velocity vectors (adata.obsm)
# Cross-Boundary Direction Correctness (A->B)
{('dividing', 'progenitors'): -0.628389363883234, ('progenitors', 'activating'): 0.6459372668797811}
Total Mean: 0.008773951498273591
# In-cluster Coherence
{'B cell lineage': 0.89785475, 'activating': 0.85710996, 'dividing': 0.8879369, 'macrophages': 0.9036216, 'progenitors': 0.8096315}
Total Mean: 0.871230959892273
# Cross-Boundary Direction Correctness (A->B)
{('dividing', 'progenitors'): -0.628389363883234, ('progenitors', 'activating'): 0.6459372668797811}
Total Mean: 0.008773951498273591
# In-cluster Coherence
{'B cell lineage': 0.89785475, 'activating': 0.85710996, 'dividing': 0.8879369, 'macrophages': 0.9036216, 'progenitors': 0.8096315}
Total Mean: 0.871230959892273
MouseErythroid
Filtered out 47456 genes t

INFO:train:Beginning training of DeepVelo_Base ...


velo data shape: torch.Size([9815, 3000])


INFO:trainer:    epoch          : 1
INFO:trainer:    time:          : 26.225547313690186
INFO:trainer:    loss           : 108466.9765625
INFO:trainer:    mse            : 187.16323852539062
INFO:trainer:    epoch          : 2
INFO:trainer:    time:          : 26.447940587997437
INFO:trainer:    loss           : 71751.90625
INFO:trainer:    mse            : 1027.203369140625
INFO:trainer:    epoch          : 3
INFO:trainer:    time:          : 26.059962034225464
INFO:trainer:    loss           : 15989.021484375
INFO:trainer:    mse            : 139.50559997558594
INFO:trainer:    epoch          : 4
INFO:trainer:    time:          : 26.21686577796936
INFO:trainer:    loss           : 16661.314453125
INFO:trainer:    mse            : 193.2812042236328
INFO:trainer:    epoch          : 5
INFO:trainer:    time:          : 26.336792945861816
INFO:trainer:    loss           : 26056.5546875
INFO:trainer:    mse            : 384.08770751953125
INFO:trainer:    epoch          : 6
INFO:trainer: 